# Einfache Abfrage

_Weitere Informationen sind im Code in den mit # markierten Zeilen zu finden_

### Import 
import verschiedener Module wie z.B. zum Zeitmessen  

In [1]:
import pathlib
import csv
import time

## Spezifikation für das Funktionieren in der Cloud
Die folgenden Codefelder werden nur im bereich der Cloud gebraucht und entfallen bei benutzung auf einem lokalen System 

An den Stellen an welchen die Punkte '.........' sind werden eigentlich die erforderlichen Infos eingesetzt, welche aber aus sicherheitstechnischen Gründen von mir entfernt wurden 

In [2]:
# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
cgsCreds = {
    'IBM_API_KEY_ID': '........',
    'IAM_SERVICE_ID': 'crn:v1:bluemix:public:iam-identity::a/a6c0651c3aab4e74a4d1e131c1858b21::serviceid:ServiceId-cf18b8e2-291a-4175-b02f-4fa012271ab7',
    'ENDPOINT': 'https://s3.private.eu-de.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': '........',
    'BUCKET': '........'
}

In [3]:
from ibm_botocore.client import Config
import ibm_boto3

def read_file_cos(credentials,local_file_name, key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    data_cos = None 
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')


In [4]:
read_file_cos(cgsCreds, "/var/tmp/tmp.csv", "2017-01-02_sds011_sensor_299.csv")


File Downloaded


## Die Eingabe
die individuelle Eingabe

In [5]:
# die folgenden Zahlen bei y,m,d und s sind im Ramen der Verfügbarkeit frei wählbar
y = 2019                #year
m = 4                   #month
d = 5                  #day
s = 299                 #sensor

filename = "%04d-%02d-%02d_sds011_sensor_%d.csv"  %(y,m,d,s)
path = "/var/tmp/" + filename
read_file_cos(cgsCreds, path, filename)

# path ist der Pfad zu der Datei, in welcher die Rohdaten liegen 

print(path)

File Downloaded
/var/tmp/2019-04-05_sds011_sensor_299.csv


## Der eigentliche Code

In [6]:
sPM2_5 = []
sPM10 = []

In [7]:
# Start der Zeit für die Zeitmessung
start = time.time()

with open(path,newline = '') as f:
    readers = csv.DictReader(f, delimiter= ';')
    for row in readers:
        PM10 = float(row["P1"])
        PM2_5 = float(row["P2"])
        sPM10.append(PM10)
        sPM2_5.append(PM2_5)
        pass
      

# Anzahl der PM2_5 und PM10 Werte
anzAll = 0

# Summe aller addierten PM10 Werte
wertAllPM10 = 0


for i in range(0, len(sPM10)):
        
    #PM10 
    wertAllPM10 += sPM10[i]
    anzAll += 1

# Summe aller addierten PM2_5 Werte
wertAllPM2_5 = 0

for i in range(0, len(sPM2_5)):
        
    #PM2_5
    wertAllPM2_5 += sPM2_5[i]

# Durchschnittswert = Summe aller Werte durch Anzahl aller Werte
durchWertPM10 = wertAllPM10 / anzAll
durchWertPM2_5 = wertAllPM2_5 / anzAll

# Ende der Zeit für die Zeitmessung
end = time.time()

# endzeit - Anfangszeit = benötigte Zeit
diff = end-start



## Die Printouts

In [8]:
print("Der Feinstaubwert (PM10) lag am %d.%d.%d bei %.2f μm pro m^3" %(d,m,y,durchWertPM10))
print("Der Feinstaubwert (PM2_5) lag am %d.%d.%d bei %.2f μm pro m^3" %(d,m,y,durchWertPM2_5))


print("Sec: %.4f" %(diff)) 


Der Feinstaubwert (PM10) lag am 5.4.2019 bei 9.50 μm pro m^3
Der Feinstaubwert (PM2_5) lag am 5.4.2019 bei 7.39 μm pro m^3
Sec: 0.0034
